In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'  # Replace with your file path
new_data = pd.read_csv(file_path)

# Example mapping (replace this with your actual mapping)
zipcode_to_neighborhood = {
    '02108': 'Beacon Hill',
    '02109': 'North End',
    # Add other mappings
}

# Function to fill missing neighborhood values
def fill_neighborhood(row):
    if pd.isna(row['Neighborhood']) and row['Zipcode'] in zipcode_to_neighborhood:
        return zipcode_to_neighborhood[row['Zipcode']]
    else:
        return row['Neighborhood']

# Apply the function to the DataFrame
new_data['Neighborhood'] = new_data.apply(fill_neighborhood, axis=1)


<ipython-input-1-ae6375351e54>:5: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv(file_path)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'
data = pd.read_csv(file_path)

# Drop rows where specified columns have NaN values
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
data = data.dropna(subset=columns_to_check)

# Drop specified columns
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']
data = data.drop(columns=columns_to_drop)

# Create a mapping from existing data for Neighborhood
zipcode_neighborhood_map = data.dropna(subset=['Zipcode', 'Neighborhood']).groupby('Zipcode')['Neighborhood'].agg(pd.Series.mode).to_dict()

# Function to fill missing neighborhood values
def fill_neighborhood(row):
    if pd.isna(row['Neighborhood']) and row['Zipcode'] in zipcode_neighborhood_map:
        return zipcode_neighborhood_map[row['Zipcode']]
    else:
        return row['Neighborhood']

# Applying the function to fill missing Neighborhood values
data['Neighborhood'] = data.apply(fill_neighborhood, axis=1)

# Selecting features and target variable
features = ['Property Type', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
            'Minimum Stay', 'Number of Photos', 'host_is_superhost_in_period', 'numReviews_pastYear',
            'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear', 'available_days', 'booked_days',
            'booked_days_avePrice', 'Nightly Rate', 'occupancy_rate', 'City_x', 'Neighborhood', 'Zipcode']
X = data[features]
y = data['Rating Overall']  # Ensure this is the correct target column name

# Preprocessing for categorical and numerical features
numerical_features = ['Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
                      'Minimum Stay', 'Number of Photos', 'numReviews_pastYear',
                      'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                      'available_days', 'booked_days', 'booked_days_avePrice',
                      'Nightly Rate', 'occupancy_rate']
categorical_features = ['Property Type', 'City_x', 'Neighborhood', 'Zipcode',
                        'host_is_superhost_in_period']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

# Create a pipeline with 2-factor interaction polynomial features and a linear regression model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('poly_features', PolynomialFeatures(interaction_only=True, include_bias=False)),
                           ('regressor', LinearRegression())])

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fitting the model
pipeline.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 164.9073720752416
R-squared: 0.22185312001728374


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'
data = pd.read_csv(file_path)

# Dropping rows and columns as per your instructions
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']

data = data.dropna(subset=columns_to_check).drop(columns=columns_to_drop)

# Selecting features and target variable
features = ['Property Type', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
            'Minimum Stay', 'Number of Photos', 'host_is_superhost_in_period', 'numReviews_pastYear',
            'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear', 'available_days', 'booked_days',
            'booked_days_avePrice', 'Nightly Rate', 'occupancy_rate', 'City_x', 'Neighborhood', 'Zipcode']
X = data[features]
y = data['Rating Overall']  # Ensure this is the correct target column name

# Preprocessing for categorical and numerical features
numerical_features = ['Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
                      'Minimum Stay', 'Number of Photos', 'numReviews_pastYear',
                      'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                      'available_days', 'booked_days', 'booked_days_avePrice',
                      'Nightly Rate', 'occupancy_rate']
categorical_features = ['Property Type', 'City_x', 'Neighborhood', 'Zipcode',
                        'host_is_superhost_in_period']

# Define transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Preprocessing the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    sparse_threshold=0)  # Ensuring the output is a dense matrix

X_processed = preprocessor.fit_transform(X)
y = y.values  # Ensure y is a numpy array

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Get feature names from the preprocessor
def get_feature_names(column_transformer):
    feature_names = []

    for transformer in column_transformer.transformers_[:-1]:  # Last one is remainder and not used
        transformed_names = transformer[1].get_feature_names_out(input_features=transformer[2])
        feature_names.extend(transformed_names)

    return feature_names

feature_names = get_feature_names(preprocessor)

# Function for forward selection
def forward_selection(X, y, feature_names, significance_level=0.05):
    initial_features = list(range(X.shape[1]))
    best_features_indices = []

    while len(initial_features) > 0:
        remaining_features = list(set(initial_features) - set(best_features_indices))
        new_pval = pd.Series(dtype=float)

        for new_column in remaining_features:
            current_features = best_features_indices + [new_column]
            model = sm.OLS(y, sm.add_constant(X[:, current_features])).fit()
            p_values = model.pvalues[1:]  # Exclude the constant term
            new_pval.at[feature_names[new_column]] = p_values[-1]  # Get the p-value of the last added feature

        min_p_value = new_pval.min()
        if min_p_value < significance_level:
            best_features_indices.append(new_pval.idxmin())
        else:
            break

    return [feature_names[i] for i in best_features_indices]

# Apply forward selection
selected_features = forward_selection(X_train, y_train, feature_names)

# Fit the model with selected features
selected_feature_indices = [feature_names.index(f) for f in selected_features]
model = sm.OLS(y_train, sm.add_constant(X_train[:, selected_feature_indices])).fit()

# Print the summary of the model
print(model.summary())


IndexError: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'
data = pd.read_csv(file_path)

# Drop rows where specified columns have NaN values
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
data = data.dropna(subset=columns_to_check)

# Drop specified columns
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']
data = data.drop(columns=columns_to_drop)

# Create a mapping from existing data for Neighborhood
zipcode_neighborhood_map = data.dropna(subset=['Zipcode', 'Neighborhood']).groupby('Zipcode')['Neighborhood'].agg(pd.Series.mode).to_dict()

# Function to fill missing neighborhood values
def fill_neighborhood(row):
    if pd.isna(row['Neighborhood']) and row['Zipcode'] in zipcode_neighborhood_map:
        return zipcode_neighborhood_map[row['Zipcode']]
    else:
        return row['Neighborhood']

# Applying the function to fill missing Neighborhood values
data['Neighborhood'] = data.apply(fill_neighborhood, axis=1)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'
data = pd.read_csv(file_path)

# Dropping rows and columns as per your instructions
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']

data = data.dropna(subset=columns_to_check).drop(columns=columns_to_drop)

# Selecting features and target variable
features = ['Property Type', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
            'Minimum Stay', 'Number of Photos', 'host_is_superhost_in_period', 'numReviews_pastYear',
            'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear', 'available_days', 'booked_days',
            'booked_days_avePrice', 'Nightly Rate', 'occupancy_rate', 'City_x', 'Neighborhood', 'Zipcode']
X = data[features]
y = data['Rating Overall']  # Ensure this is the correct target column name

# Preprocessing for categorical and numerical features
numerical_features = ['Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
                      'Minimum Stay', 'Number of Photos', 'numReviews_pastYear',
                      'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                      'available_days', 'booked_days', 'booked_days_avePrice',
                      'Nightly Rate', 'occupancy_rate']
categorical_features = ['Property Type', 'City_x', 'Neighborhood', 'Zipcode',
                        'host_is_superhost_in_period']

# Define transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Preprocessing the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    sparse_threshold=0)  # Ensuring the output is a dense matrix

X_processed = preprocessor.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Fitting a Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Predicting and evaluating the model
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Training Mean Squared Error: {train_mse}')
print(f'Training R-squared: {train_r2}')
print(f'Testing Mean Squared Error: {test_mse}')
print(f'Testing R-squared: {test_r2}')


Training Mean Squared Error: 7.102234942832142
Training R-squared: 0.9663406218862001
Testing Mean Squared Error: 47.22202835814408
Testing R-squared: 0.7771738548075379


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'
data = pd.read_csv(file_path)

# Dropping rows and columns as per your instructions
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']

data = data.dropna(subset=columns_to_check).drop(columns=columns_to_drop)

# Selecting features and target variable
features = ['Property Type', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
            'Minimum Stay', 'Number of Photos', 'host_is_superhost_in_period', 'numReviews_pastYear',
            'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear', 'available_days', 'booked_days',
            'booked_days_avePrice', 'Nightly Rate', 'occupancy_rate', 'City_x', 'Neighborhood', 'Zipcode']
X = data[features]
y = data['Rating Overall']

# Preprocessing for categorical and numerical features
numerical_features = ['Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
                      'Minimum Stay', 'Number of Photos', 'numReviews_pastYear',
                      'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                      'available_days', 'booked_days', 'booked_days_avePrice',
                      'Nightly Rate', 'occupancy_rate']
categorical_features = ['Property Type', 'City_x', 'Neighborhood', 'Zipcode',
                        'host_is_superhost_in_period']

# Define transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Preprocessing the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    sparse_threshold=0)

X_processed = preprocessor.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Create a Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Randomized search on hyper parameters
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
rf_random.fit(X_train, y_train)

# Best parameters
print("Best Parameters: ", rf_random.best_params_)

# Using the best estimator found by RandomizedSearchCV
best_rf = rf_random.best_estimator_

# Predictions and evaluation
y_train_pred = best_rf.predict(X_train)
y_test_pred = best_rf.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Optimized Training Mean Squared Error: {train_mse}')
print(f'Optimized Training R-squared: {train_r2}')
print(f'Optimized Testing Mean Squared Error: {test_mse}')
print(f'Optimized Testing R-squared: {test_r2}')

# Feature Importance
importances = best_rf.feature_importances_
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False)
print(feature_importances)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'
data = pd.read_csv(file_path)

# Dropping rows and columns as per your instructions
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']

data = data.dropna(subset=columns_to_check).drop(columns=columns_to_drop)

# Selecting features and target variable
features = ['Property Type', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
            'Minimum Stay', 'Number of Photos', 'host_is_superhost_in_period', 'numReviews_pastYear',
            'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear', 'available_days', 'booked_days',
            'booked_days_avePrice', 'Nightly Rate', 'occupancy_rate', 'City_x', 'Neighborhood', 'Zipcode']
X = data[features]
y = data['Rating Overall']

# Preprocessing for categorical and numerical features
numerical_features = ['Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
                      'Minimum Stay', 'Number of Photos', 'numReviews_pastYear',
                      'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                      'available_days', 'booked_days', 'booked_days_avePrice',
                      'Nightly Rate', 'occupancy_rate']
categorical_features = ['Property Type', 'City_x', 'Neighborhood', 'Zipcode',
                        'host_is_superhost_in_period']

# Define transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Preprocessing the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    sparse_threshold=0)

X_processed = preprocessor.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Initialize models
rf_model = RandomForestRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)
ridge_model = Ridge(random_state=42)

# Neural Network Model
nn_model = Sequential()
nn_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dense(1))
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Fit models
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)
nn_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

# Predictions
rf_pred = rf_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)
ridge_pred = ridge_model.predict(X_test)
nn_pred = nn_model.predict(X_test).flatten()

# Average the predictions
ensemble_pred = (rf_pred + xgb_pred + ridge_pred + nn_pred) / 4

# Evaluate the ensemble model
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
ensemble_r2 = r2_score(y_test, ensemble_pred)

print(f'Ensemble Model Mean Squared Error: {ensemble_mse}')
print(f'Ensemble Model R-squared: {ensemble_r2}')


427/427 [==============================] - 1s 1ms/step
Ensemble Model Mean Squared Error: 63.98856292292871
Ensemble Model R-squared: 0.6980577643894775


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'  # Update this with your file path
data = pd.read_csv(file_path)

# Dropping rows and columns as per your instructions
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']

data = data.dropna(subset=columns_to_check).drop(columns=columns_to_drop)

# Selecting features and target variable
features = ['Property Type', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
            'Minimum Stay', 'Number of Photos', 'host_is_superhost_in_period', 'numReviews_pastYear',
            'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear', 'available_days', 'booked_days',
            'booked_days_avePrice', 'Nightly Rate', 'occupancy_rate', 'City_x', 'Neighborhood', 'Zipcode']
X = data[features]
y = data['Rating Overall']

# Preprocessing for categorical and numerical features
numerical_features = ['Bedrooms', 'Bathrooms', 'Max Guests', 'Cleaning Fee (USD)',
                      'Minimum Stay', 'Number of Photos', 'numReviews_pastYear',
                      'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                      'available_days', 'booked_days', 'booked_days_avePrice',
                      'Nightly Rate', 'occupancy_rate']
categorical_features = ['Property Type', 'City_x', 'Neighborhood', 'Zipcode',
                        'host_is_superhost_in_period']

# Define transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Preprocessing the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    sparse_threshold=0)

X_processed = preprocessor.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Hyperparameter tuning for RandomForest and XGBoost
param_grid_rf = {'n_estimators': [100, 200], 'max_depth': [10, 20]}
param_grid_xgb = {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1]}

rf = RandomForestRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)

rf_grid = GridSearchCV(rf, param_grid_rf, cv=3, n_jobs=-1)
xgb_grid = GridSearchCV(xgb_model, param_grid_xgb, cv=3, n_jobs=-1)

rf_grid.fit(X_train, y_train)
xgb_grid.fit(X_train, y_train)

# Neural Network Model
nn_model = Sequential()
nn_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
nn_model.add(Dropout(0.3))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dropout(0.3))
nn_model.add(Dense(1))
nn_model.compile(optimizer='adam', loss='mean_squared_error')
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Stacking Ensemble
estimators = [
    ('rf', rf_grid.best_estimator_),
    ('xgb', xgb_grid.best_estimator_)
]
stack_reg = StackingRegressor(estimators=estimators, final_estimator=Ridge())
stack_reg.fit(X_train, y_train)

# Predictions and evaluation
y_pred_stack = stack_reg.predict(X_test)
ensemble_mse = mean_squared_error(y_test, y_pred_stack)
ensemble_r2 = r2_score(y_test, y_pred_stack)

print(f'Stacking Ensemble Mean Squared Error: {ensemble_mse}')
print(f'Stacking Ensemble R-squared: {ensemble_r2}')


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = 'AirBnb Pre processed.csv'
data = pd.read_csv(file_path)

# Load the dataset
file_path = '/content/AirBnb Pre processed.csv'  # Update this with your file path
data = pd.read_csv(file_path)

# Dropping rows and columns as per your instructions
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']

data = data.dropna(subset=columns_to_check).drop(columns=columns_to_drop)

# Assuming the target variable is 'rating_overall'
y = data['Rating Overall']
X = data.drop('Rating Overall', axis=1)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Preprocessing for categorical and numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)])

# Split the dataset into training and testing sets
X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Initialize models
rf_model = RandomForestRegressor(random_state=42)
gb_model = GradientBoostingRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Fit models
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Neural Network Model
nn_model = Sequential()
nn_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
nn_model.add(Dropout(0.3))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dropout(0.3))
nn_model.add(Dense(1))
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10)
nn_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stop], verbose=1)

# Predictions and evaluation
rf_pred = rf_model.predict(X_test)
gb_pred = gb_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)
nn_pred = nn_model.predict(X_test).flatten()

# Averaging predictions
ensemble_pred = (rf_pred + gb_pred + xgb_pred + nn_pred) / 4

# Evaluate the ensemble model
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
ensemble_r2 = r2_score(y_test, ensemble_pred)

print(f'Ensemble Model Mean Squared Error: {ensemble_mse}')
print(f'Ensemble Model R-squared: {ensemble_r2}')


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = 'AirBnb Pre processed.csv'
data = pd.read_csv(file_path)

# Create a mapping from existing data for Neighborhood
zipcode_neighborhood_map = data.dropna(subset=['Zipcode', 'Neighborhood']).groupby('Zipcode')['Neighborhood'].agg(pd.Series.mode).to_dict()

# Function to fill missing neighborhood values
def fill_neighborhood(row):
    if pd.isna(row['Neighborhood']) and row['Zipcode'] in zipcode_neighborhood_map:
        return zipcode_neighborhood_map[row['Zipcode']]
    else:
        return row['Neighborhood']

# Applying the function to fill missing Neighborhood values
data['Neighborhood'] = data.apply(fill_neighborhood, axis=1)

# Dropping rows and columns as per your instructions
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']

# Assuming the target variable is 'rating_overall'
y = data['Rating Overall']
X = data.drop('Rating Overall', axis=1)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Preprocessing for categorical and numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)])

X_processed = preprocessor.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

# Initialize models
rf_model = RandomForestRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Fit models
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Predictions and evaluation
rf_pred = rf_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

# Averaging predictions
ensemble_pred = (rf_pred + xgb_pred) / 2

# Evaluate the ensemble model
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
ensemble_r2 = r2_score(y_test, ensemble_pred)

print(f'Ensemble Model Mean Squared Error: {ensemble_mse}')
print(f'Ensemble Model R-squared: {ensemble_r2}')


ValueError: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = '/content/AirBnb Pre processed Tableau 1.csv'
data = pd.read_csv(file_path)

# Drop rows where specified columns have NaN values
columns_to_check = ['rating_ave_pastYear', 'numReviews_pastYear', 'numCancel_pastYear',
                    'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
                    'prev_rating_ave_pastYear', 'prev_numReviews_pastYear',
                    'prev_numCancel_pastYear', 'prev_num_5_star_Rev_pastYear',
                    'prev_prop_5_StarReviews_pastYear', 'Property Type']
data = data.dropna(subset=columns_to_check)

# Drop specified columns
columns_to_drop = ['Nightly Rate_tractQuartile', 'prev_Nightly Rate_tractQuartile',
                   'available_days_aveListedPrice_tractQuartile',
                   'prev_available_days_aveListedPrice_tractQuartile']
data = data.drop(columns=columns_to_drop)

# Create a mapping from existing data for Neighborhood
zipcode_neighborhood_map = data.dropna(subset=['Zipcode', 'Neighborhood']).groupby('Zipcode')['Neighborhood'].agg(pd.Series.mode).to_dict()

# Function to fill missing neighborhood values
def fill_neighborhood(row):
    if pd.isna(row['Neighborhood']) and row['Zipcode'] in zipcode_neighborhood_map:
        return zipcode_neighborhood_map[row['Zipcode']]
    else:
        return row['Neighborhood']

# Applying the function to fill missing Neighborhood values
data['Neighborhood'] = data.apply(fill_neighborhood, axis=1)

# Import necessary libraries

import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv(file_path)

# Calculate Occupancy Rate
df['occupancy_rate'] = df['booked_days'] / (df['booked_days'] + df['available_days'])

# Selecting relevant variables
features = ['Cleaning Fee (USD)', 'host_is_superhost_in_period', 'prev_host_is_superhost_in_period',
            'rating_ave_pastYear', 'numReviews_pastYear', 'num_5_star_Rev_pastYear',
            'prop_5_StarReviews_pastYear', 'numReservedDays_pastYear', 'numReserv_pastYear',
            'numCancel_pastYear', 'available_days', 'available_days_aveListedPrice',
            'booked_days', 'booked_days_avePrice', 'occupancy_rate']

# Assuming 'revenue' is your target variable
target = 'revenue'

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Model training and evaluation
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Lasso': Lasso(),
    'Ridge': Ridge()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(f'{name} - RMSE: {np.sqrt(mean_squared_error(y_test, predictions))}, R2: {r2_score(y_test, predictions)}')


KeyError: ignored

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
df = pd.read_csv('/content/AirBnb Pre processed Tableau 1.csv')

# Selecting relevant variables (replace with exact names as needed)
features = ['Cleaning Fee (USD)', 'Host Is Superhost In Period', 'rating ave pastYear', 'numReviews pastYear',
            'Booked Days', 'available days aveListedPrice', 'Nightly Rate']

# Assuming 'Revenue' and 'Occupancy Rate' are your target variables
y_revenue = df['Revenue']
y_occupancy_rate = df['Occupancy Rate']


# Prepare the data
X = data[features]
y = data[target]

# Handle missing values if needed
X.fillna(X.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)
# Train-test split for both targets
X_train_revenue, X_test_revenue, y_train_revenue, y_test_revenue = train_test_split(df[features], y_revenue, test_size=0.2, random_state=42)
X_train_occupancy, X_test_occupancy, y_train_occupancy, y_test_occupancy = train_test_split(df[features], y_occupancy_rate, test_size=0.2, random_state=42)


# Function to train and evaluate models
def train_and_evaluate(models, X_train, X_test, y_train, y_test, target_name):
    for name, model in models.items():
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        print(f'{name} - {target_name} - RMSE: {np.sqrt(mean_squared_error(y_test, predictions))}, R2: {r2_score(y_test, predictions)}')

# Models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Lasso': Lasso(),
    'Ridge': Ridge()
}

# Evaluating models for Revenue
print("Evaluating models for Revenue:")
train_and_evaluate(models, X_train_revenue, X_test_revenue, y_train_revenue, y_test_revenue, "Revenue")

# Evaluating models for Occupancy Rate
print("\nEvaluating models for Occupancy Rate:")
train_and_evaluate(models, X_train_occupancy, X_test_occupancy, y_train_occupancy, y_test_occupancy, "Occupancy Rate")


KeyError: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, MaxAbsScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.pipeline import Pipeline
 
# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
 
# Load the dataset
data = pd.read_csv('/content/AirbnbHouston_Preprocessed_dataset 2.csv')  # Adjust path as needed
 
# Define the target variable and features
target = 'occupancy_rate'
features = ['host_is_superhost_in_period', 'numReviews_pastYear', 'booked_days',
            'rating_ave_pastYear', 'num_5_star_Rev_pastYear', 'prop_5_StarReviews_pastYear',
            'prev_rating_ave_pastYear', 'prev_prop_5_StarReviews_pastYear', 'Minimum Stay',
            'Max Guests', 'Zipcode', 'Pets Allowed', 'available_days', 'Nightly Rate',
            'Cleaning Fee (USD)', 'Bedrooms', 'Bathrooms', 'Rating Overall', 'superhost_change_lose_superhost','superhost_change_gain_superhost']
 
X = data[features]
y = data[target]
 
# Splitting the dataset into training and validation sets (60:40)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)
 
# Preprocessing
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MaxAbsScaler())])
 
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)])
 
# Fit and transform training data
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
 
# Create polynomial features
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train_processed)
X_val_poly = poly_features.transform(X_val_processed)
 
# Linear Regression Model
linear_reg = LinearRegression()
linear_reg.fit(X_train_poly, y_train)
 
# Predict and evaluate on training and validation data
y_train_pred = linear_reg.predict(X_train_poly)
y_val_pred = linear_reg.predict(X_val_poly)
 
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
val_mse = mean_squared_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)
 
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
 
# Print model summary
print("Model Summary:")
 
print("Training MSE:", train_mse)
print("Training R-squared:", train_r2)
print("Training MAPE:", train_mape, "%")
print("Validation MSE:", val_mse)
print("Validation R-squared:", val_r2)
print("Validation MAPE:", val_mape, "%")
 
print("Intercept:", linear_reg.intercept_)
print("Coefficients:", linear_reg.coef_)